In [1]:
import os
import json
import requests
import pandas as pd
import re

### Configuration Files

In [2]:
MAIN_DIR = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
PLAYERS_JSON = os.path.join(MAIN_DIR, "players.json")
GIRLS_PLAYERS_CSV = os.path.join(MAIN_DIR, "girls_players.csv")
SCHOOLS_CSV = os.path.join(MAIN_DIR, "schools.csv")

PLAYERS_URL = "https://api.v2.tennisreporting.com/players"

### Helper Functions

In [ ]:
# --------------------------------------------------
# Helpers
# --------------------------------------------------

def clean_lastname(name: str) -> str:
    """Clean last names like '\"Lastname,\"' → 'Lastname'."""
    if pd.isna(name):
        return name
    s = str(name).strip()
    # Remove surrounding quotes
    s = s.strip('"\'')
    # Remove trailing commas
    s = s.rstrip(',')
    return s


def can_encode_utf8(value) -> bool:
    """Return False if the value cannot be encoded as UTF-8."""
    if pd.isna(value):
        return True
    try:
        str(value).encode("utf-8")
        return True
    except UnicodeEncodeError:
        return False



### Extract player data from tennisreporting.com

In [4]:
# Fetch players JSON and store as players.json
print("Downloading player data...")
resp = requests.get(PLAYERS_URL)
resp.raise_for_status()
players_data = resp.json()

print(f"Writing JSON to {PLAYERS_JSON} ...")
with open(PLAYERS_JSON, "w", encoding="utf-8") as f:
    json.dump(players_data, f, ensure_ascii=False, indent=2)
print("JSON file complete")

Writing JSON to C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\players.json ...


### Transform and Load to girls_players.csv

In [9]:
# --------------------------------------------------
# Main ETL
# --------------------------------------------------

def main():
    # 1) Read JSON into DataFrame
    print("Reading players.json into DataFrame...")
    with open(PLAYERS_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    df = pd.DataFrame(data)

    # 2) Load schools.csv and get valid schoolIDs
    print(f"Reading schools from {SCHOOLS_CSV} ...")
    schools = pd.read_csv(SCHOOLS_CSV, dtype={"SchoolID": "Int64"})
    valid_school_ids = set(schools["SchoolID"].dropna().astype(int))

    # 3) Clean last names
    print("Cleaning lastName values...")
    df["lastName"] = df["lastName"].apply(clean_lastname)
    
    # 4) Clean leading/trailing whitespace from firstName and lastName
    print("Stripping whitespace from firstName and lastName...")
    df["firstName"] = df["firstName"].astype(str).str.strip()
    df["lastName"]  = df["lastName"].astype(str).str.strip()

    # 5) Create fullname = firstname + ' ' + lastname
    print("Creating fullname field...")
    df["fullName"] = df["firstName"].fillna("").str.strip() + " " + df["lastName"].fillna("").str.strip()
    df["fullName"] = df["fullName"].str.strip()

    # 6) Filtering according to rules

    # 6.1) Only include players whose schoolId is present in schools.csv
    before = len(df)
    df = df[df["schoolId"].isin(valid_school_ids)]
    print(f"Filtered on schoolId in schools.csv: {before} -> {len(df)} rows")

    # 6.2) Only include records where genderId = 2
    before = len(df)
    df = df[df["genderId"] == 2]
    print(f"Filtered on genderId == 2: {before} -> {len(df)} rows")

    # 6.3) Exclude records where firstname=Default and lastname=Default
    before = len(df)
    mask_default = (df["firstName"] == "Default") & (df["lastName"] == "Default")
    df = df[~mask_default]
    print(f"Excluded Default/Default names: {before} -> {len(df)} rows")

    # 6.4) Exclude records where graduatedDate is NOT null and year < 2025
    #      Keep rows where graduatedDate is null OR year >= 2025
    before = len(df)
    grad_dates = pd.to_datetime(df["graduatedDate"], errors="coerce", utc=True)
    keep_mask = grad_dates.isna() | (grad_dates.dt.year >= 2025)
    df = df[keep_mask]
    print(f"Filtered on graduatedDate (exclude < 2025): {before} -> {len(df)} rows")

    # 7) Select and rename columns for output CSV
    #    Only keep: id, schoolID, firstname, lastname, grade, fullname
    print("Selecting and renaming columns for girls_players.csv...")
    df_out = df[["id", "schoolId", "firstName", "lastName", "grade", "fullName"]].copy()
    df_out = df_out.rename(
        columns={
            "id": "playerID",
            "schoolId": "schoolID",
            "firstName": "firstname",
            "lastName": "lastname",
            "fullName": "fullname",
        }
    )

    # 8) Write girls_players.csv
    print(f"Writing {GIRLS_PLAYERS_CSV} ...")
    df_out.to_csv(GIRLS_PLAYERS_CSV, index=False, encoding="utf-8")
    print("girls_players.csv written successfully.")

if __name__ == "__main__":
    main()

print("Transform and Load Complete")

Reading players.json into DataFrame...
Reading schools from C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.csv ...
Cleaning lastName values...
Stripping whitespace from firstName and lastName...
Creating fullname field...
Filtered on schoolId in schools.csv: 139612 -> 9614 rows
Filtered on genderId == 2: 9614 -> 5365 rows
Excluded Default/Default names: 5365 -> 5341 rows
Filtered on graduatedDate (exclude < 2025): 5341 -> 2928 rows
Selecting and renaming columns for girls_players.csv...
Transform and Load Complete


In [15]:
# --- Read CSV ---
df = pd.read_csv(GIRLS_PLAYERS_CSV)

# --- Print first 10 records ---
print("\n=== TOP 10 RECORDS ===")
print(df.head(10).to_string(index=False))

# --- Print last 10 records ---
print("\n=== LAST 10 RECORDS ===")
print(df.tail(10).to_string(index=False))



=== TOP 10 RECORDS ===
 playerID  schoolID firstname   lastname     grade          fullname
    80635   75619.0     Halle    Shearer Graduated     Halle Shearer
    80636   75619.0    Hannah    Millkey Graduated    Hannah Millkey
    81005   74665.0    Sophie     Qualls Graduated     Sophie Qualls
    81697   75554.0    Sidrah    Schramm        12    Sidrah Schramm
    98157   75744.0    Hailey      Foley Graduated      Hailey Foley
    98158   75744.0     Vicky     Nguyen Graduated      Vicky Nguyen
    98682   75550.0     Maddy    Naugler Graduated     Maddy Naugler
    99025   74728.0    Isabel Cunningham Graduated Isabel Cunningham
    99026   74728.0    Olivia     Jacoby Graduated     Olivia Jacoby
    99031   74614.0     Laina     Atiyeh Graduated      Laina Atiyeh

=== LAST 10 RECORDS ===
 playerID  schoolID firstname lastname     grade        fullname
   172901   75129.0     Keely    Burns        10     Keely Burns
   172920   75448.0     Eleem     Reta Graduated      Eleem Re

### Data Quality Checks

In [14]:
from function_scripts.data_quality import check_csv

results = check_csv(
    csv_path=GIRLS_PLAYERS_CSV,
    unique_fields=["playerID"],         
    null_fields=["playerID","firstname","lastname","fullname", "schoolID","grade"],
    whitespace_fields=["firstname","lastname","fullname","grade"]
)


===== DATA QUALITY CHECKS =====

Reading CSV: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\girls_players.csv
Loaded 2928 rows.

Checking for duplicate values...

✅ No duplicates in 'playerID'

---------------------------------------

Checking for null values...

✅ No null values in 'playerID'
✅ No null values in 'firstname'
✅ No null values in 'lastname'
✅ No null values in 'fullname'
✅ No null values in 'schoolID'
✅ No null values in 'grade'

---------------------------------------

Checking for leading/trailing whitespace...

⚠️ Field 'playerID' is not a string. Cannot check whitespace.
✅ No leading/trailing whitespace in 'firstname'
✅ No leading/trailing whitespace in 'lastname'
✅ No leading/trailing whitespace in 'fullname'
⚠️ Field 'schoolID' is not a string. Cannot check whitespace.
✅ No leading/trailing whitespace in 'grade'

===== DATA QUALITY CHECK COMPLETE =====



### Custom Data Quality Checks

In [16]:
# Create the dataframe
df_out = pd.read_csv(GIRLS_PLAYERS_CSV)

print("\n--- CUSTOM DATA QUALITY CHECKS ---")

# 1) Show all firstname values that have non-alphabetic characters (excluding spaces)
#    (A-Z or a-z only)
fname_nonalpha_mask = df_out["firstname"].astype(str).str.contains(r"[^A-Za-z ]", na=False)
bad_firstnames = df_out.loc[fname_nonalpha_mask, "firstname"].dropna().unique()
print("\n2) Firstnames with non-alphabetic characters:")
if len(bad_firstnames) == 0:
    print("   None found.")
else:
    for name in sorted(bad_firstnames):
        print("  ", name)

# 2) Show all lastname values that have non-alphabetic characters (excluding spaces and hyphens)
lname_nonalpha_mask = df_out["lastname"].astype(str).str.contains(r"[^A-Za-z -]", na=False)
bad_lastnames = df_out.loc[lname_nonalpha_mask, "lastname"].dropna().unique()
print("\n3) Lastnames with non-alphabetic characters:")
if len(bad_lastnames) == 0:
    print("   None found.")
else:
    for name in sorted(bad_lastnames):
        print("  ", name)

# 3) Check for values that will cause utf-8 encoding problems
print("\n4) Checking for UTF-8 encoding issues...")
problem_cells = []
for col in df_out.columns:
    if df_out[col].dtype == "object":
        for idx, val in df_out[col].items():
            if not can_encode_utf8(val):
                problem_cells.append((idx, col, repr(val)))

if not problem_cells:
    print("   No UTF-8 encoding problems detected.")
else:
    print("   Potential UTF-8 encoding problems found in the following cells:")
    for idx, col, val_repr in problem_cells:
        print(f"   Row {idx}, Column {col}, Value {val_repr}")

# 4) Frequency report on first name and show 5 most common
print("\n5) Top 5 most common firstnames:")
freq_first = df_out["firstname"].value_counts().head(5)
print(freq_first)


--- DATA QUALITY CHECKS ---

2) Firstnames with non-alphabetic characters:
   Alexa G.
   Alijah (Jordan)
   Ana Rocio (Rosie)
   Asha B.
   Azul. N
   Carolina (Zoe)
   Eden (Alex)
   Elena (Sophia)
   Han (Hana)
   Ina M.
   Jamie. M
   Jenna-Rose
   Jenny. G
   Katherine (Katy)
   Kaydence. EF
   Maylanie (Lani)
   Maëlys
   Nicole (Nikki)
   Quetzalli. C
   Rissa-Unique
   Samatha`
   Ta'Resia
   Yoselin. Y
   default2

3) Lastnames with non-alphabetic characters:
   .Lambourne
   Castañon
   O'Connell
   Saldaña
   Shepard (Barry)
   St. Amand
   St. Aubin
   St.Clair
   Zamora Hernández
   default2

4) Checking for UTF-8 encoding issues...
   No UTF-8 encoding problems detected.

5) Top 5 most common firstnames:
firstname
Olivia      32
Sophia      28
Emily       24
Ella        24
Isabella    24
Name: count, dtype: int64


### Identify players with multiple IDs (same first and last name and schoolID)
### Write them to players_with_multiple_ids.csv so they can be consolidated in the match files or ratings

In [17]:
# --- File paths ---
main_dir = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
csv_path = os.path.join(main_dir, "girls_players.csv")
output_path = os.path.join(main_dir, "players_with_multiple_ids.csv")

# --- Load file ---
df = pd.read_csv(csv_path, dtype={"playerID": int, "schoolID": int})

# --- Normalize name fields (recommended) ---
df["firstname"] = df["firstname"].astype(str).str.strip()
df["lastname"]  = df["lastname"].astype(str).str.strip()

# --- Group by firstname, lastname, schoolID ---
group_cols = ["firstname", "lastname", "schoolID"]
duplicate_groups = df.groupby(group_cols)

mapping_dict = {}

# --- Process each group ---
for group_key, group_df in duplicate_groups:
    if len(group_df) > 1:  # Only groups with duplicates
        max_id = group_df["playerID"].max()
        for pid in group_df["playerID"]:
            if pid != max_id:
                mapping_dict[pid] = max_id

# --- Print result ---
print("\n=== PlayerID Mapping (duplicates → max ID) ===")
if mapping_dict:
    for old_id, new_id in mapping_dict.items():
        print(f"{old_id} -> {new_id}")
else:
    print("No players with multiple IDs detected.")

# --- Save to CSV ---
if mapping_dict:
    df_out = pd.DataFrame(list(mapping_dict.items()), columns=["old_playerID", "new_playerID"])
    df_out.to_csv(output_path, index=False)
    print(f"\nMapping saved to: {output_path}")
else:
    print("\nNo mapping file created (no duplicates found).")


=== PlayerID Mapping (duplicates → max ID) ===
100525 -> 128882
143598 -> 169128
124185 -> 168386
122976 -> 124727
162497 -> 162498
172920 -> 172923
172921 -> 172923
172922 -> 172923
142431 -> 164432
143170 -> 170256
143167 -> 171924
145890 -> 172536
124836 -> 145256
149397 -> 171518
161963 -> 162345
143164 -> 171912
142662 -> 151837
121943 -> 172535
122814 -> 171922
121440 -> 122799
161965 -> 162348
121381 -> 141693
163455 -> 163456
141281 -> 171591
121069 -> 163885
106325 -> 143050
161989 -> 168713
122284 -> 143338
122806 -> 171917
168688 -> 168697
143165 -> 171911

Mapping saved to: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\players_with_multiple_ids.csv
